This code will eventually be used to generate sequence for the compartments used in the SyNa model. So far it does not work, but one day!

In [ ]:
import subprocess
import os
from pathlib import Path

# Paths
LM_DESIGN_DIR = "/home/shd-sun-lab/SynapseNavigator/esm/examples/lm-design"
REPO_ROOT = "/home/shd-sun-lab/SynapseNavigator"

# Base environment – make sure SynapseNavigator is on PYTHONPATH so `protgps` imports work
base_env = os.environ.copy()
base_env["PYTHONPATH"] = REPO_ROOT + ":" + base_env.get("PYTHONPATH", "")

def run_guided_free_generation(
    seed: int,
    class_idx: str,
    cuda_device_idx: int = 0,
    num_iter: int = 1000,
):
    """
    Run one free_generation job using guided_lm_design.py.

    Parameters
    ----------
    seed : int
        Random seed for the MCMC / generation.
    class_idx : str
        Target compartment label. MUST match one of CLASSIFIER_CLASSES in guided_lm_design.py:
            "cytosol", "ER", "mitochondrion",
            "nucleus", "Excitatory Synapse", "Inhibitory Synapses"
    cuda_device_idx : int
        GPU index. Currently ignored because we force CPU via disable_cuda=True.
    num_iter : int
        Number of MCMC steps for free generation (Hydra cfg: tasks.free_generation.num_iter).
    """

    # Hydra override for class_idx – quote it because it contains a space
    class_override = f"+tasks.fixedbb.accept_reject.energy_cfg.class_idx='{class_idx}'"

    num_iter_override = f"tasks.free_generation.num_iter={num_iter}"

    cmd = [
        "python",
        "guided_lm_design.py",
        "task=free_generation",
        f"seed={seed}",
        f"cuda_device_idx={cuda_device_idx}",

        f"+tasks.fixedbb.accept_reject.energy_cfg.class_idx='{class_idx}'",

        num_iter_override,          # <-- FIXED HERE

        "+disable_cuda=True",
        "+save_path=dummy",
        "+attachment=start",
        '+init_sequence=""',
        "+classifier_threshold=0.95",
        "+classifier_lag=10",
        "+classifier_consistency_steps=5",
        "+warmup_steps=100",
        "+exit_at_success=True",
    ]


    print(f"\nRunning: {' '.join(cmd)}")
    result = subprocess.run(
        cmd,
        cwd=LM_DESIGN_DIR,
        env=base_env,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
    )
    print(result.stdout)
    print("Return code:", result.returncode)


In [54]:
run_guided_free_generation(
    seed=1,
    class_idx="Excitatory Synapse",
    cuda_device_idx=0,
    num_iter=1000,   # you can bump this later
)



Running: python guided_lm_design.py task=free_generation seed=1 cuda_device_idx=0 +tasks.fixedbb.accept_reject.energy_cfg.class_idx='Excitatory Synapse' tasks.free_generation.num_iter=1000 +disable_cuda=True +save_path=dummy +attachment=start +init_sequence="" +classifier_threshold=0.95 +classifier_lag=10 +classifier_consistency_steps=5 +warmup_steps=100 +exit_at_success=True
Could not append to config. An item is already at 'disable_cuda'.
Either remove + prefix: 'disable_cuda=True'
Or add a second + to add or override 'disable_cuda': '++disable_cuda=True'


Set the environment variable HYDRA_FULL_ERROR=1 for a complete stack trace.

Return code: 1
